# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:42679 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.36 GB


## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [2]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/travis/build/dask/dask-examples/data/0.json',
 '/home/travis/build/dask/dask-examples/data/1.json',
 '/home/travis/build/dask/dask-examples/data/2.json',
 '/home/travis/build/dask/dask-examples/data/3.json',
 '/home/travis/build/dask/dask-examples/data/4.json',
 '/home/travis/build/dask/dask-examples/data/5.json',
 '/home/travis/build/dask/dask-examples/data/6.json',
 '/home/travis/build/dask/dask-examples/data/7.json',
 '/home/travis/build/dask/dask-examples/data/8.json',
 '/home/travis/build/dask/dask-examples/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module.

In [3]:
!head -n 2 data/0.json

{"age": 55, "name": ["Lizeth", "Tucker"], "occupation": "Panel Beater", "telephone": "(679) 120-9827", "address": {"address": "1250 Belcher Nene", "city": "Hartford"}, "credit-card": {"number": "3445 965131 42044", "expiration-date": "09/20"}}
{"age": 28, "name": ["Thomas", "Hammond"], "occupation": "Make Up Artist", "telephone": "182.766.7904", "address": {"address": "1157 Northridge Place", "city": "Rancho Santa Margarita"}, "credit-card": {"number": "2599 6853 7972 2847", "expiration-date": "11/25"}}


In [4]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [5]:
b.take(2)

({'age': 55,
  'name': ['Lizeth', 'Tucker'],
  'occupation': 'Panel Beater',
  'telephone': '(679) 120-9827',
  'address': {'address': '1250 Belcher Nene', 'city': 'Hartford'},
  'credit-card': {'number': '3445 965131 42044', 'expiration-date': '09/20'}},
 {'age': 28,
  'name': ['Thomas', 'Hammond'],
  'occupation': 'Make Up Artist',
  'telephone': '182.766.7904',
  'address': {'address': '1157 Northridge Place',
   'city': 'Rancho Santa Margarita'},
  'credit-card': {'number': '2599 6853 7972 2847',
   'expiration-date': '11/25'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [6]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 55,
  'name': ['Lizeth', 'Tucker'],
  'occupation': 'Panel Beater',
  'telephone': '(679) 120-9827',
  'address': {'address': '1250 Belcher Nene', 'city': 'Hartford'},
  'credit-card': {'number': '3445 965131 42044', 'expiration-date': '09/20'}},
 {'age': 41,
  'name': ['Donny', 'Craft'],
  'occupation': 'Circus Proprietor',
  'telephone': '717.125.7158',
  'address': {'address': '528 Dorman Bay', 'city': 'Greenfield'},
  'credit-card': {'number': '5308 6231 5311 8234',
   'expiration-date': '03/24'}})

In [7]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Panel Beater', 'Make Up Artist')

In [8]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [9]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [10]:
result.compute()

[('Lighting Designer', 15),
 ('Transport Clerk', 15),
 ('Radio Controller', 15),
 ('Planning Manager', 14),
 ('Bus Conductor', 14),
 ('Tea Blender', 14),
 ('Underwriter', 13),
 ('Parts Man', 13),
 ('Parts Supervisor', 13),
 ('Chauffeur', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [11]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/travis/build/dask/dask-examples/data/processed.0.json',
 '/home/travis/build/dask/dask-examples/data/processed.1.json',
 '/home/travis/build/dask/dask-examples/data/processed.2.json',
 '/home/travis/build/dask/dask-examples/data/processed.3.json',
 '/home/travis/build/dask/dask-examples/data/processed.4.json',
 '/home/travis/build/dask/dask-examples/data/processed.5.json',
 '/home/travis/build/dask/dask-examples/data/processed.6.json',
 '/home/travis/build/dask/dask-examples/data/processed.7.json',
 '/home/travis/build/dask/dask-examples/data/processed.8.json',
 '/home/travis/build/dask/dask-examples/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [12]:
b.take(1)

({'age': 55,
  'name': ['Lizeth', 'Tucker'],
  'occupation': 'Panel Beater',
  'telephone': '(679) 120-9827',
  'address': {'address': '1250 Belcher Nene', 'city': 'Hartford'},
  'credit-card': {'number': '3445 965131 42044', 'expiration-date': '09/20'}},)

In [13]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 55,
  'occupation': 'Panel Beater',
  'telephone': '(679) 120-9827',
  'credit-card-number': '3445 965131 42044',
  'credit-card-expiration': '09/20',
  'name': 'Lizeth Tucker',
  'street-address': '1250 Belcher Nene',
  'city': 'Hartford'},)

In [14]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,55,Panel Beater,(679) 120-9827,3445 965131 42044,09/20,Lizeth Tucker,1250 Belcher Nene,Hartford
1,28,Make Up Artist,182.766.7904,2599 6853 7972 2847,11/25,Thomas Hammond,1157 Northridge Place,Rancho Santa Margarita
2,23,Merchant Seaman,712.775.0496,3471 114793 23351,02/25,Christinia Odom,212 Hancock Drive,St. Charles
3,41,Circus Proprietor,717.125.7158,5308 6231 5311 8234,03/24,Donny Craft,528 Dorman Bay,Greenfield
4,65,Word Processing Operator,695-329-8979,4719 1288 2684 4923,06/20,Norman Sexton,1093 Jerrold Bypass,St. Louis


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [15]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

Lighting Designer    15
Radio Controller     15
Transport Clerk      15
Tea Blender          14
Bus Conductor        14
Planning Manager     14
Parts Supervisor     13
Chauffeur            13
Pasteuriser          13
Parts Man            13
Name: occupation, dtype: int64

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](http://docs.dask.org/en/latest/bag-overview.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)
-  [dask tutorial](https://github.com/dask/dask-tutorial), notebook 02, for a more in-depth introduction.